# Revista de Administração e Contabilidade da FAT

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(query, url_search, pg_num=1):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    tbl_results = soup.find('table', {'class': 'listing'})
    a_links = []
    for a in tbl_results.find_all('a'):
           if 'Resumo' in a.text:
                a_links.append(a['href'])
    return a_links



In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(query, url_search, pg_num)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links



In [5]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [6]:
def captura_titulo(soup):
    try:
        title_tag = soup.find('div', {'id': 'articleTitle'})
        title = title_tag.find('h3').text
    except:
        title = ''
    return {'Título': title}

In [7]:
def captura_ano(soup):
    try:
        year_tags = soup.find('div', {'id': 'breadcrumb'})
        year_tag = year_tags.find_all('a')
        year_text = year_tag[1].text
        year = year_text[-5:-1]
    except:
        year = ''
    return {'Data de Publicação': year}

In [8]:
def captura_resumo(soup):
    try:
        abstract_tag = soup.find('div', {'id': 'articleAbstract'})
        abstract = abstract_tag.find('div').text
    except:
        abstract = ''
    return {'Resumo': abstract}

In [9]:
def captura_palavras_chave(soup):
    try:
        kw_tag = soup.find('div', {'id': 'articleSubject'})
        kw = kw_tag.find('div').text.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [10]:
def captura_autores(soup):
    autores = []
    try:
        authors_tag = soup.find('div', {'id': 'authorString'})
        autores = authors_tag.find('em').text.split(', ')
        
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [11]:
def captura_afiliacao(soup):
    affiliation_list = []
    try:
        authors_tag = soup.find_all('div', {'class': 'authorBio'})
        for p in authors_tag:
            try:
                affiliation = p.find_all('br')[-2].next_sibling
                affiliation = affiliation.replace('\t', '')
            except:
                affiliation = ''
            affiliation_list.append(affiliation)
    except:
        pass
    affiliation_list = completar_lista(affiliation_list)
    key_affiliation = [f"Afilição {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, affiliation_list))

In [12]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista de Administração e Contabilidade da FAT'}
    url_search_text = 'http://www.reacfat.com.br/index.php/reac/search/search?query=__query__&searchJournal=1&authors=&title=&abstract=&galleyFullText=&suppFiles=&discipline=&subject=&type=&coverage=&indexTerms=&dateFromMonth=&dateFromDay=&dateFromYear=2016&dateToMonth=&dateToDay=&dateToYear=2019&orderBy=&orderDir=&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_ano(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y')
    return df

## Pesquisa por artigos

In [13]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,Url Artigo,Revista
0,2018-01-01,Perfil e evolução da pesquisa em gestão de res...,O objetivo desta pesquisa foi investigar as ca...,,5,Alison Silva Ferreira,Mirian Gomes Conceição,Sonia Maria da Silva Gomes,Lucas Welter dos Santos Ribeiro,Ruthyeli de Jesus Ribeiro,,,,,,,,http://www.reacfat.com.br/index.php/reac/artic...,Revista de Administração e Contabilidade da FAT
1,2018-01-01,Aderência à Matriz Curricular proposta pela Fu...,Este trabalho é resultante de uma pesquisa que...,,3,Lucivânia Cordeiro Silva,Juliano Almeida de Faria,Laerson Morais Silva Lopes,,,,,,,,,,http://www.reacfat.com.br/index.php/reac/artic...,Revista de Administração e Contabilidade da FAT
2,2012-01-01,Transparência Fiscal nos Municípios do Cone Su...,­­­­­­­­­­­A transparência das contas da gestã...,Transparência. Gestão pública. Municípios. Ron...,2,Alexandre de Freitas Carneiro,Luzia Cestaro Gambarini,,,,,,,,,,,http://www.reacfat.com.br/index.php/reac/artic...,Revista de Administração e Contabilidade da FAT


In [14]:
df.to_csv('../data/ReAC.csv', index=False)

In [15]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,Url Artigo,Revista
0,2018-01-01,Perfil e evolução da pesquisa em gestão de res...,O objetivo desta pesquisa foi investigar as ca...,,5,Alison Silva Ferreira,Mirian Gomes Conceição,Sonia Maria da Silva Gomes,Lucas Welter dos Santos Ribeiro,Ruthyeli de Jesus Ribeiro,,,,,,,,http://www.reacfat.com.br/index.php/reac/artic...,Revista de Administração e Contabilidade da FAT
1,2018-01-01,Aderência à Matriz Curricular proposta pela Fu...,Este trabalho é resultante de uma pesquisa que...,,3,Lucivânia Cordeiro Silva,Juliano Almeida de Faria,Laerson Morais Silva Lopes,,,,,,,,,,http://www.reacfat.com.br/index.php/reac/artic...,Revista de Administração e Contabilidade da FAT
2,2012-01-01,Transparência Fiscal nos Municípios do Cone Su...,­­­­­­­­­­­A transparência das contas da gestã...,Transparência. Gestão pública. Municípios. Ron...,2,Alexandre de Freitas Carneiro,Luzia Cestaro Gambarini,,,,,,,,,,,http://www.reacfat.com.br/index.php/reac/artic...,Revista de Administração e Contabilidade da FAT
3,2014-01-01,A Atuação Profissional e a Situação Socioeconô...,Uma vez que as decisões das empresas influenci...,,1,Neylane dos Santos Oliveira,,,,,,,,,,,,http://www.reacfat.com.br/index.php/reac/artic...,Revista de Administração e Contabilidade da FAT
4,2014-01-01,A Atuação Profissional e a Situação Socioeconô...,Uma vez que as decisões das empresas influenci...,,1,Neylane dos Santos Oliveira,,,,,,,,,,,,http://www.reacfat.com.br/index.php/reac/artic...,Revista de Administração e Contabilidade da FAT
5,2016-01-01,"Créditos de Carbono, Contabilidade e Projetos ...",Questões ambientais têm sido temas emergentes ...,,2,Alexandre de Freitas Carneiro,Luciano Bezerra da Silva,,,,,,,,,,,http://www.reacfat.com.br/index.php/reac/artic...,Revista de Administração e Contabilidade da FAT
6,2019-01-01,Legitimidade Corporativa em Empresas Ambiental...,O presente estudo buscou verificar se as empre...,,2,Gabrielle Pompeu Sodré,Antonio Gualberto Pereira,,,,,,,,,,,http://www.reacfat.com.br/index.php/reac/artic...,Revista de Administração e Contabilidade da FAT
